# Part I. ETL Pipeline for Pre-Processing the Files

## RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion. 

## CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries you we want to run.

# Create queries to ask the following three questions of the data



### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4




In [8]:
## Create table based on the query requirment and declaring PK and CK
query = "CREATE TABLE IF NOT EXISTS song_info_per_item_session "
query = query + """ (
                    sessionId INT,
                    itemInSession INT,
                    artist_name TEXT,
                    song_name TEXT,
                    length FLOAT,
                    PRIMARY KEY (sessionId, itemInSession)
                    )"""
try:
    session.execute(query)
except Exception as e:
    print(e)

### 1.2  Set up the CSV file and INSERT statments into the query variable



In [9]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_per_item_session (sessionId, itemInSession, artist_name, song_name, length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### 1.3 Use SELECT to verify that the data have been inserted into each table

In [10]:
## use SELECT to get data from table created
query = """
SELECT 
    artist_name, 
    song_name, 
    length
FROM
    song_info_per_item_session
WHERE
    sessionId = 338
AND
    itemInSession = 4
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Faithless', song_name='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
## Create table based on the query requirment and declaring PK and CK

query = "CREATE TABLE IF NOT EXISTS song_artist_by_session "
query = query + """ (
                    userId INT,
                    sessionId INT,
                    itemInSession INT,
                    artist_name TEXT,
                    song_name TEXT,
                    firstName TEXT,
                    lastName TEXT,
                    PRIMARY KEY ((userId, sessionId), itemInSession)
                    )"""
try:
    session.execute(query)
except Exception as e:
    print(e)         

### 2.2  Set up the CSV file and INSERT statments into the query variable




In [12]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_artist_by_session (userId, sessionId, itemInSession, artist_name, song_name, firstName, lastName)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### 2.3 Use SELECT to verify that the data have been inserted into each table

In [13]:
## Use SELECT to get data from table created

query = """
SELECT 
    artist_name, 
    song_name, 
    firstName,
    lastName
FROM
    song_artist_by_session
WHERE
    sessionId = 182
AND
    userId = 10
ORDER BY
    itemInSession
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist_name='Down To The Bone', song_name="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist_name='Three Drives', song_name='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Sebastien Tellier', song_name='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Lonnie Gordon', song_name='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [17]:
## Create table based on the query requirment and declaring PK and CK

query = "CREATE TABLE IF NOT EXISTS name_info_by_song_name "
query = query + """(
                    song_name TEXT,
                    firstName TEXT,
                    lastName TEXT,
                    userId INT,
                    PRIMARY KEY (song_name, userId))
                """
try:
    session.execute(query)
except Exception as e:
    print(e)         

                    

### 3.2  Set up the CSV file and INSERT statments into the query variable



In [18]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO name_info_by_song_name (song_name, firstName, lastName, userId)"
        query = query + "VALUES(%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

### 3.3 Use SELECT to verify that the data have been inserted into each table

In [19]:
## Use SELECT to get data from table created

query = """
SELECT 
    firstName, 
    lastName
FROM 
    name_info_by_song_name
WHERE 
    song_name='All Hands Against His Own';
"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)


Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [16]:
session.execute("drop table song_info_per_item_session")
session.execute("drop table song_artist_by_session")
session.execute("drop table name_info_by_song_name")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()